In [ ]:
import numpy as np
import random
import operator
import pandas as pd
from scipy.special import loggamma 
import math

**Genetic algorithm**

**Pseudocode**  
1. start
2. generate initial population
3. fitness  
4-1. selection  
4-2. crossover  
4-3. mutation  
4-4. fitness  


In [ ]:
M = np.array(pd.read_csv('opt3.txt', header = None, delimiter = " "))

In [ ]:
def shuffle(n, m=20):
  orders = []
  for i in range(n):
    order = np.array(range(m))
    random.shuffle(order)
    orders.append(order)
  return orders

In [ ]:
def prob(M, order):
  p = 0.0
  for i in range(1, len(order)):
    sl = np.array(2 * M[:, order[i - 1]] + M[:, order[i]])
    ns = [sum(sl == j) for j in range(0, 4)]
    p -= loggamma(ns[0] + ns[1] + 1/2) + loggamma(ns[2] + ns[3] + 1/2)
    p += sum(map(lambda n: loggamma(n + 1/4), ns))
  return p

In [ ]:
# xa and yb orders

def crossover(parents_orders):
  children = []
  x, a = parents_orders[0][0:10], parents_orders[0][10:21]
  y, b = parents_orders[1][0:10], parents_orders[1][10:21]
  child_1 = np.concatenate((x, b))
  child_2 = np.concatenate((y, a))
  children.append(child_1)
  children.append(child_2)
  return(children)

In [ ]:
def mutation(order):
  random_index = random.randint(0, 18)
  x1 = order[random_index]
  x2 = order[random_index+1]
  order[random_index], order[random_index+1] = x2, x1
  return order

In [86]:
def genetic_algorithm(M):
  orders = shuffle(100) #get 100 arrats with positions 20
  for iter in range(10):
    logs = []
    for i in range(len(orders)):
      log = prob(M, orders[i])
      logs.append(log)
  
    #selection
    number_of_selection = math.floor(0.8 * len(logs))
    if number_of_selection % 2 != 0:
      number_of_selection += 1
    best_rows = np.argpartition(logs, -number_of_selection)[-number_of_selection:]
    selected = []
    for num in best_rows:
      selected.append(orders[num])
    
    #crossover
    i = 0
    children = []
    while i < len(selected):
      parents = []
      parents.append(selected[i])
      i += 1
      parents.append(selected[i])
      i += 1
      childs = crossover(parents)
      for ch in childs:
        children.append(ch)

    #mutation
    post_mutation = []
    for c in children:
      mutated = mutation(c)
      post_mutation.append(mutated)

    #second fitness
    new_logs = []
    for i in range(len(post_mutation)):
      new_log = prob(M, post_mutation[i])
      new_logs.append(new_log)
    
    if max(logs) <= max(new_logs):
      orders = post_mutation
      
  print(min(logs))
  min_log = min(logs)
  ind = logs.index(min_log)
  print(orders[ind])


In [88]:
genetic_algorithm(M)

-1098.1159778765195
[12 15  8 18  4  5 13  2 14 16  1  6 10  3  7 11  2  8 19 14]


**Annealing**

In [12]:
import math

In [82]:
def annealing(M, order):
  temp = 0.1
  p_current = prob(M, order)
  p_best = prob(M, order)
  s_current = order
  s_best = order
  for iter in range(8):
    s_new = np.array(range(20))
    random.shuffle(s_new)
    p_new = prob(M, s_new)

    difference = p_new - p_current
    temp = round(temp/float(iter+1), 6)
    mac = math.exp(-difference / temp)

    if difference < 0 or random.random() < mac:
      s_current = s_new
      p_current = p_new

    if p_new < p_best:
      s_best = s_new
      p_best = p_new

  print(p_best)
  print(s_best)

In [85]:
order = np.array(range(20))
random.shuffle(order)

annealing(M, order)

-1046.8275628793049
[13 14  0  9  4 12  2 11 10 19  5  6  1 17  8 15  7  3 18 16]
